In [1]:
import json
import numpy as np
from sentence_transformers import SentenceTransformer
from googletrans import Translator
from langdetect import detect
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
import json

# Load the JSONL dataset
def load_dataset(file_path):
    questions = []
    answers = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            entry = json.loads(line)
            questions.append(entry['prompt'])
            answers.append(entry['completion'])
    return questions, answers

# Load the data
questions, answers = load_dataset(r'C:\Users\HP\Downloads\agriculture_ques\Bigdata.jsonl')


In [3]:
import re

# Simple regex to detect language based on script characters
def detect_language(text):
    if re.search(r'[a-zA-Z]', text):
        return 'en'  # English
    elif re.search(r'[\u0900-\u097F]', text):
        return 'hi'  # Hindi (Devanagari script)
    elif re.search(r'[\u0A00-\u0A7F]', text):
        return 'pa'  # Punjabi (Gurmukhi script)
    else:
        return 'en'  # Default to English if undetermined


In [13]:
import json
from googletrans import Translator
from time import sleep

# Initialize Translator
translator = Translator()

# Load the data from the JSONL file
with open(r'C:\Users\HP\Downloads\agriculture_ques\Bigdata.jsonl', 'r', encoding='utf-8') as file:
    data = [json.loads(line) for line in file]

# Initialize dictionaries to store translations
translations = {
    'en': {},
    'hi': {},
    'pa': {}
}

# Function to translate a batch of texts
def translate_batch(texts, src_lang, dest_lang):
    translations = []
    for text in texts:
        try:
            translated_text = translator.translate(text, src=src_lang, dest=dest_lang).text
            translations.append(translated_text)
        except Exception as e:
            print(f"Error translating text: {text[:50]}... - {e}")
            translations.append("")  # Append empty string on error
            sleep(1)  # Pause briefly to reduce likelihood of rate limiting
    return translations

# Process each entry in the data
for entry in data:
    question = entry['prompt']
    answer = entry['completion']

    # Store original English texts
    translations['en'][question] = answer

    # Translate the question to Hindi and Punjabi
    hindi_question_translation = translate_batch([question], src_lang='en', dest_lang='hi')[0]
    punjabi_question_translation = translate_batch([question], src_lang='en', dest_lang='pa')[0]

    # Translate the answer to Hindi and Punjabi
    hindi_answer_translation = translate_batch([answer], src_lang='en', dest_lang='hi')[0]
    punjabi_answer_translation = translate_batch([answer], src_lang='en', dest_lang='pa')[0]

    # Store translations in the format: {question: answer}
    translations['hi'][hindi_question_translation] = hindi_answer_translation
    translations['pa'][punjabi_question_translation] = punjabi_answer_translation

    # Intermediate save every 50 entries
    if len(translations['hi']) % 50 == 0:
        with open('translations_partial.json', 'w', encoding='utf-8') as file:
            json.dump(translations, file, ensure_ascii=False, indent=4)
        print(f"Saved partial results for {len(translations['hi'])} entries...")

# Save final translations to a JSON file
with open('translations.json', 'w', encoding='utf-8') as file:
    json.dump(translations, file, ensure_ascii=False, indent=4)

print("Translations saved to 'translations.json'")


Saved partial results for 50 entries...
Saved partial results for 100 entries...
Saved partial results for 150 entries...
Saved partial results for 200 entries...
Saved partial results for 250 entries...
Saved partial results for 300 entries...
Saved partial results for 350 entries...
Saved partial results for 400 entries...
Saved partial results for 450 entries...
Saved partial results for 500 entries...
Saved partial results for 550 entries...
Saved partial results for 600 entries...
Saved partial results for 650 entries...
Saved partial results for 650 entries...
Saved partial results for 700 entries...
Saved partial results for 750 entries...
Saved partial results for 800 entries...
Saved partial results for 850 entries...
Saved partial results for 900 entries...
Saved partial results for 950 entries...
Saved partial results for 1000 entries...
Saved partial results for 1050 entries...
Saved partial results for 1100 entries...
Saved partial results for 1150 entries...
Saved partial

KeyboardInterrupt: 

In [5]:
import json

# Load translations from JSON file
with open('translations.json', 'r', encoding='utf-8') as file:
    translations = json.load(file)
# Simple string matching based on word overlap
def find_best_response(query, questions, answers, user_language):
    # Get the corresponding questions in the user language
    localized_questions = translations[user_language].keys() if user_language in translations else questions

    best_match = None
    max_overlap = 0
    best_answer = None
    
    # Find the question with the most overlapping words
    query_words = set(query.split())
    for i, q in enumerate(localized_questions):
        question_words = set(q.split())
        overlap = len(query_words & question_words)  # Word overlap count
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_match = q
            best_answer = translations[user_language][q] if user_language in translations and q in translations[user_language] else answers[i]

    # Default response if no match found
    if not best_answer:
        best_answer = "I'm sorry, I couldn't understand that. Can you try rephrasing?"
    return best_answer


In [7]:
import json

# Load the JSONL dataset
def load_dataset(file_path):
    questions = []
    answers = []
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            entry = json.loads(line)
            questions.append(entry['prompt'])
            answers.append(entry['completion'])
    return questions, answers

def chatbot():
    questions, answers = load_dataset(r'C:\Users\HP\Downloads\agriculture_ques\Bigdata.jsonl')
    print("Welcome to the Farmer's Chatbot! Type 'exit' to quit.")

    while True:
        user_query = input("\nYou: ")
        
        if user_query.lower() == 'exit':
            print("Bot: Goodbye!")
            break

        # Detect language and find best response
        user_language = detect_language(user_query)
        response = find_best_response(user_query, questions, answers, user_language)
        print("Bot:", response)

# Run chatbot
chatbot()


Welcome to the Farmer's Chatbot! Type 'exit' to quit.
Bot: I'm sorry, I couldn't understand that. Can you try rephrasing?
Bot: Goodbye!


In [1]:
import json
import re

# Load translations from JSON file
with open('translations.json', 'r', encoding='utf-8') as file:
    translations = json.load(file)

# Language detection based on script characters
def detect_language(text):
    if re.search(r'[a-zA-Z]', text):
        return 'en'  # English
    elif re.search(r'[\u0900-\u097F]', text):
        return 'hi'  # Hindi (Devanagari script)
    elif re.search(r'[\u0A00-\u0A7F]', text):
        return 'pa'  # Punjabi (Gurmukhi script)
    else:
        return 'en'  # Default to English if undetermined

# Normalize text by converting to lowercase and removing any special characters
def normalize_text(text):
    return re.sub(r'\W+', ' ', text.lower()).strip()

# Function to find the closest matching question in the user's language
def find_best_response(query, user_language):
    # Check if language exists in translations; if not, default to English
    if user_language not in translations:
        print(f"Language '{user_language}' not found in translations; defaulting to English.")
        user_language = 'en'
    
    # Get localized questions and answers for the detected language
    localized_questions = list(translations[user_language].keys())
    localized_answers = translations[user_language]

    best_match = None
    max_overlap = 0
    best_answer = None

    # Normalize the user's query for better matching
    query_words = set(normalize_text(query).split())
    
    # Find the question with the most overlapping words
    for q in localized_questions:
        question_words = set(normalize_text(q).split())
        overlap = len(query_words & question_words)
        
        if overlap > max_overlap:
            max_overlap = overlap
            best_match = q
            best_answer = localized_answers[q]

    if not best_answer:
        best_answer = "I'm sorry, I couldn't understand that. Can you try rephrasing?"
    return best_answer

# Chatbot interface
def chatbot():
    print("Welcome to the Farmer's Chatbot! Type 'exit' to quit.")

    while True:
        user_query = input("\nYou: ")
        print("You:",user_query)
        
        if user_query.lower() == 'exit':
            print("Bot: Goodbye!")
            break

        # Detect language and find the best response
        user_language = detect_language(user_query)
        response = find_best_response(user_query, user_language)
        print("Bot:", response)

# Run chatbot
chatbot()


Welcome to the Farmer's Chatbot! Type 'exit' to quit.
You: ਟਮਾਟਰ ਨੂੰ ਦੇਰ ਨਾਲ ਕਿਵੇਂ ਭੰਡਾਰ ਕਰਨਾ ਹੈ?
Bot: ਟਮਾਟਰ ਦੇਰ ਨਾਲ ਝੁਲਸ ਇੱਕ ਆਮ ਬਿਮਾਰੀ ਹੈ ਜੋ ਟਮਾਟਰ ਦੇ ਪੌਦਿਆਂ ਨੂੰ ਬਹੁਤ ਨੁਕਸਾਨ ਪਹੁੰਚਾ ਸਕਦੀ ਹੈ.ਦੇਰ ਨਾਲ ਝੁਲਸਣ ਲਈ, ਤੁਸੀਂ ਹੇਠ ਦਿੱਤੇ ਕਦਮ ਚੁੱਕ ਸਕਦੇ ਹੋ:
1. ਇਨਡੋਫਿਲ ਐਮ -45 ਦੀ ਵਰਤੋਂ ਕਰੋ ਇਨੋਫਿਲ ਐਮ -45 ਵਿਚ ਇਨਡੋਫਿਲ ਐਮ -5 ਪ੍ਰਤੀ ਲੀਟਰ ਪਾਣੀ ਅਤੇ ਸੰਕਰਮਿਤ ਪੌਦਿਆਂ ਦੇ 2.5 ਗ੍ਰਾਮ ਸ਼ਾਮਲ ਕਰੋ.ਇਹ ਏਜੰਟ ਜਰਾਸੀਮਾਂ ਦੇ ਵਾਧੇ ਨੂੰ ਰੋਕ ਸਕਦਾ ਹੈ, ਜਿਸ ਨਾਲ ਬਿਮਾਰੀ ਦੀ ਗੰਭੀਰਤਾ ਨੂੰ ਘਟਾ ਸਕਦਾ ਹੈ.
2. ਲਾਗ ਵਾਲੇ ਪੌਦਿਆਂ ਨੂੰ ਤੁਰੰਤ ਹਟਾਓ: ਇਕ ਵਾਰ ਸੰਕਰਮਿਤ ਪੌਦਿਆਂ ਨੂੰ ਮਿਲਾਉਣ ਲਈ ਤੁਰੰਤ ਹਟਾਓ ਅਤੇ ਨਸ਼ਟ ਕਰੋ.
3. ਸਪਰੇਅ ਫੰਜਾਈਗਾਈਡਜ਼: ਫੰਜਾਈਗਾਈਡਾਈਡਜ਼ ਦੀ ਨਿਯਮਤ ਛਿੜਕਾਅ, ਜਿਵੇਂ ਕਿ ਡਿਵਿਨੋ ਕੌਨਲ ਜਾਂ ਮੰਗਨੀਜ਼ ਓਕਸਾਈਡ ਅਸਰਦਾਰ ਤਰੀਕੇ ਨਾਲ ਬਿਮਾਰੀ ਦੇ ਵਿਕਾਸ ਨੂੰ ਨਿਯੰਤਰਿਤ ਕਰ ਸਕਦਾ ਹੈ.
4. ਪੌਦੇ ਹਵਾਦਾਰ ਰੱਖੋ: ਇਹ ਸੁਨਿਸ਼ਚਿਤ ਕਰੋ ਕਿ ਹਵਾਦਾਰੀ ਨੂੰ ਉਤਸ਼ਾਹਤ ਕਰਨ ਅਤੇ ਨਮੀ ਨੂੰ ਘਟਾਉਣ ਲਈ ਪੌਦਿਆਂ ਵਿਚਕਾਰ ਕਾਫ਼ੀ ਜਗ੍ਹਾ ਹੈ, ਜਿਸ ਨਾਲ ਕੀਟਾਣੂਆਂ ਨੂੰ ਫੈਲਾਉਣ ਦੇ ਜੋਖਮ ਨੂੰ ਘਟਾਉਂਦਾ ਹੈ.
ਇਹ ਕਦਮ ਚੁੱਕਣ ਨਾਲ, ਤੁਸੀਂ ਟਮਾਟਰ ਦੇਰ ਨਾਲ ਝੁਲਸਣ ਅਤੇ ਆਪਣੇ ਟਮਾਟਰ ਦੇ ਪੌਦਿਆਂ ਦੀ ਸਿਹਤ ਨੂੰ ਸੁਰੱਖਿਅਤ ਕਰ ਸਕਦੇ ਹੋ.
You: quit
Bot: I'm sorry, I couldn'